### DART API 이용

전자공시시스템(DART ; Data Analysis, Retrieval and Transfer System)은 상장법인 등이 공시서류를 인터넷으로 제출
하고, 투자자 등 이용자는 제출 즉시 인터넷을 통해 조회할 수 있도록 하는 종합적 기업공시 시스템입니다.


* [상장 법인 목록](https://kind.krx.co.kr/corpgeneral/corpList.do?method=loadInitPage)
* [API 개발 가이드](https://dart.fss.or.kr/dsap001/guide.do)

DART 크롤링 관련 포스팅 <br>
https://kwonkyo.tistory.com/95

## 1. 검색 API

In [1]:
import pandas as pd
from bs4 import BeautifulSoup
from urllib.request import urlopen
import webbrowser

In [50]:
df_cd = pd.read_excel("상장법인목록.xlsx")
df_cd.head(2)

,회사명,종목코드,업종,주요제품,상장일,결산월,대표자명,홈페이지,지역
0,아이티엠반도체,84850,전자부품 제조업,배터리 보호회로,2019-11-07,12월,나혁휘,NaN,충청북도
1,미래에셋대우스팩4호,333430,금융 지원 서비스업,기업인수합병,2019-11-06,12월,서재일,NaN,서울특별시


In [55]:
df_cd_sub = df_cd.loc[df_cd["회사명"].str.contains("농심"), :"상장일"]
df_cd_sub

,회사명,종목코드,업종,주요제품,상장일
1349,농심홀딩스,72710,기타 금융업,"지주회사,투자자문,경영컨설팅 서비스",2003-07-30
2198,농심,4370,기타 식품 제조업,"라면,스낵,유지 제조,도매,수출입",1976-06-30


In [58]:
key = "a54fb6800ce976dc0a7d4862d0dfc72b1bcdc86f"
date_start = "19600101"
company_cd = df_cd_sub.reset_index(drop=True)["종목코드"][0]
company_cd = str(company_cd).zfill(6)
company_cd

'072710'

In [59]:
url_main = "http://dart.fss.or.kr/api/search.xml?auth={0}&crp_cd={1}&start_dt={2}&bsn_tp=A001&bsn_tp=A002&bsn_tp=A003"
url = url_main.format(key, company_cd, date_start)
url

'http://dart.fss.or.kr/api/search.xml?auth=a54fb6800ce976dc0a7d4862d0dfc72b1bcdc86f&crp_cd=072710&start_dt=19600101&bsn_tp=A001&bsn_tp=A002&bsn_tp=A003'

In [60]:
text = urlopen(url).read()
text_bs = BeautifulSoup(text, "lxml-xml")

In [61]:
text_bs

<?xml version="1.0" encoding="utf-8"?>
<result>
<err_code>000</err_code>
<err_msg>정상</err_msg>
<page_no>1</page_no>
<page_set>10</page_set>
<total_count>70</total_count>
<total_page>7</total_page>
<list>
<crp_cls>Y</crp_cls>
<crp_nm>농심홀딩스</crp_nm>
<crp_cd>072710</crp_cd>
<rpt_nm>분기보고서 (2019.09)</rpt_nm>
<rcp_no>20191114001043</rcp_no>
<flr_nm>농심홀딩스</flr_nm>
<rcp_dt>20191114</rcp_dt>
<rmk/>
</list>
<list>
<crp_cls>Y</crp_cls>
<crp_nm>농심홀딩스</crp_nm>
<crp_cd>072710</crp_cd>
<rpt_nm>반기보고서 (2019.06)</rpt_nm>
<rcp_no>20190814001996</rcp_no>
<flr_nm>농심홀딩스</flr_nm>
<rcp_dt>20190814</rcp_dt>
<rmk/>
</list>
<list>
<crp_cls>Y</crp_cls>
<crp_nm>농심홀딩스</crp_nm>
<crp_cd>072710</crp_cd>
<rpt_nm>분기보고서 (2019.03)</rpt_nm>
<rcp_no>20190515001783</rcp_no>
<flr_nm>농심홀딩스</flr_nm>
<rcp_dt>20190515</rcp_dt>
<rmk/>
</list>
<list>
<crp_cls>Y</crp_cls>
<crp_nm>농심홀딩스</crp_nm>
<crp_cd>072710</crp_cd>
<rpt_nm>사업보고서 (2018.12)</rpt_nm>
<rcp_no>20190401003386</rcp_no>
<flr_nm>농심홀딩스</flr_nm>
<rcp_dt>20190401</rcp_dt>
<r

In [62]:
te = text_bs.findAll("list")
te[0:2]

[<list>
 <crp_cls>Y</crp_cls>
 <crp_nm>농심홀딩스</crp_nm>
 <crp_cd>072710</crp_cd>
 <rpt_nm>분기보고서 (2019.09)</rpt_nm>
 <rcp_no>20191114001043</rcp_no>
 <flr_nm>농심홀딩스</flr_nm>
 <rcp_dt>20191114</rcp_dt>
 <rmk/>
 </list>, <list>
 <crp_cls>Y</crp_cls>
 <crp_nm>농심홀딩스</crp_nm>
 <crp_cd>072710</crp_cd>
 <rpt_nm>반기보고서 (2019.06)</rpt_nm>
 <rcp_no>20190814001996</rcp_no>
 <flr_nm>농심홀딩스</flr_nm>
 <rcp_dt>20190814</rcp_dt>
 <rmk/>
 </list>]

In [63]:
df = pd.DataFrame()
for t in te:
    df_sub = pd.DataFrame(([[t.crp_cls.string,
                             t.crp_nm.string,
                             t.crp_cd.string,
                             t.rpt_nm.string,
                             t.rcp_no.string,
                             t.flr_nm.string,
                             t.rcp_dt.string,
                             t.rmk.string]]))
    df = pd.concat([df, df_sub])
df.columns = ["crp_cls","crp_nm","crp_cd","rpt_nm",
              "rcp_no","flr_nm","rcp_dt","rmk"]

In [64]:
df = df.reset_index(drop = True)
df.head()

,crp_cls,crp_nm,crp_cd,rpt_nm,rcp_no,flr_nm,rcp_dt,rmk
0,Y,농심홀딩스,072710,분기보고서 (2019.09),20191114001043,농심홀딩스,20191114,None
1,Y,농심홀딩스,072710,반기보고서 (2019.06),20190814001996,농심홀딩스,20190814,None
2,Y,농심홀딩스,072710,분기보고서 (2019.03),20190515001783,농심홀딩스,20190515,None
3,Y,농심홀딩스,072710,사업보고서 (2018.12),20190401003386,농심홀딩스,20190401,연
4,Y,농심홀딩스,072710,분기보고서 (2018.09),20181114002737,농심홀딩스,20181114,None


In [65]:
# user_num = int(input("몇 번째 보고서를 확인하시겠습니까?"))
url_report = "http://dart.fss.or.kr/dsaf001/main.do?rcpNo=" + df['rcp_no'][0]
url_report
# webbrowser.open(url_user)

'http://dart.fss.or.kr/dsaf001/main.do?rcpNo=20191114001043'

## 2. 기업개황 API

In [45]:
url_company = "http://dart.fss.or.kr/api/company.json?auth={0}&crp_cd={1}"
crpcode = "004370"
url = url_company.format(key, crpcode)
url

'http://dart.fss.or.kr/api/company.json?auth=a54fb6800ce976dc0a7d4862d0dfc72b1bcdc86f&crp_cd=004370'

In [46]:
import requests
import json

In [47]:
response = requests.get(url)
text = json.loads(response.content)
text

{'err_code': '000',
 'err_msg': '정상',
 'crp_nm': '(주)농심',
 'crp_nm_e': 'NONGSHIM CO.,LTD',
 'crp_nm_i': '농심',
 'stock_cd': '004370',
 'ceo_nm': '신동원, 박 준',
 'crp_cls': 'Y',
 'crp_no': '1101110057574',
 'bsn_no': '1188103914',
 'adr': '서울특별시 동작구  여의대방로 112',
 'hm_url': 'www.nongshim.com',
 'ir_url': '',
 'phn_no': '02-820-7114',
 'fax_no': '02-820-7044',
 'ind_cd': '107',
 'est_dt': '19650918',
 'acc_mt': '12'}

In [48]:
from pandas.io.json import json_normalize
json_normalize(text)

,err_code,err_msg,crp_nm,crp_nm_e,crp_nm_i,stock_cd,ceo_nm,crp_cls,crp_no,bsn_no,adr,hm_url,ir_url,phn_no,fax_no,ind_cd,est_dt,acc_mt
0,000,정상,(주)농심,"NONGSHIM CO.,LTD",농심,004370,"신동원, 박 준",Y,1101110057574,1188103914,서울특별시 동작구 여의대방로 112,www.nongshim.com,,02-820-7114,02-820-7044,107,19650918,12
